In [28]:
import os
from glob import glob

import numpy as np
import torch
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
use_cuda = torch.cuda.is_available()

In [29]:
train_transforms = transforms.Compose([transforms.RandomRotation(60),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

image_path = 'data'

train_path = os.path.join(image_path, 'train')
val_path = os.path.join(image_path, 'valid')
test_path = os.path.join(image_path, 'test')

train_dataset = datasets.ImageFolder(train_path, train_transforms)
val_dataset = datasets.ImageFolder(val_path, train_transforms)
test_dataset = datasets.ImageFolder(test_path, test_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

loaders = {'train': train_loader, 'valid': val_loader, 'test': test_loader}

In [30]:
import torch.nn as nn
import torch.nn.functional as F

def calc_w_conv_out(conv, pool_stride = 1):
    return (((conv["W"] - conv["F"] + (2*conv["P"])) / conv["S"]) + 1) / pool_stride

conv1_w_in = 224
conv1 = {"W": conv1_w_in, "D": 3, "K": 16, "F": 7, "P": 0, "S": 7}
conv1_w_out = calc_w_conv_out(conv1)

conv2 = {"W": conv1_w_out, "D": conv1["K"], "K": 24, "F": 3, "P": 1, "S": 1}
conv2_w_out = calc_w_conv_out(conv2, 2)

conv3 = {"W": conv2_w_out, "D": conv2["K"], "K": 32, "F": 3, "P": 1, "S": 1}
conv3_w_out = calc_w_conv_out(conv3)

conv4 = {"W": conv3_w_out, "D": conv3["K"], "K": 48, "F": 3, "P": 1, "S": 1}
conv4_w_out = calc_w_conv_out(conv4, 4)

conv5 = {"W": conv4_w_out, "D": conv4["K"], "K": 56, "F": 3, "P": 1, "S": 1}
conv5_w_out = calc_w_conv_out(conv5)

conv6 = {"W": conv5_w_out, "D": conv5["K"], "K": 64, "F": 3, "P": 1, "S": 1}
conv6_w_out = calc_w_conv_out(conv6, 4)

conv7 = {"W": conv6_w_out, "D": conv6["K"], "K": 176, "F": 3, "P": 1, "S": 1}
conv7_w_out = calc_w_conv_out(conv7)

conv8 = {"W": conv7_w_out, "D": conv7["K"], "K": 192, "F": 3, "P": 1, "S": 1}
conv8_w_out = calc_w_conv_out(conv8, 2)

conv9 = {"W": conv8_w_out, "D": conv8["K"], "K": 208, "F": 3, "P": 1, "S": 1}
conv9_w_out = calc_w_conv_out(conv9)

conv10 = {"W": conv9_w_out, "D": conv9["K"], "K": 224, "F": 3, "P": 1, "S": 1}
conv10_w_out = calc_w_conv_out(conv10, 2)


conv_features_out = conv4_w_out**2 * conv4["K"]

#print(conv1_w_out, conv2_w_out, conv3_w_out, conv4_w_out, conv5_w_out, 
#      conv6_w_out, conv7_w_out, conv8_w_out, conv9_w_out, conv10_w_out, conv_features_out)

print(conv1_w_out, conv2_w_out, conv3_w_out, conv4_w_out, conv_features_out)

def make_nn_conv(conv):
    return nn.Conv2d(conv["D"], conv["K"], conv["F"], padding=conv["P"], stride=conv["S"])


# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        ## Layer 1
        self.conv1 = make_nn_conv(conv1)
        self.conv2 = make_nn_conv(conv2)
        ## Layer 2
        self.conv3 = make_nn_conv(conv3)
        self.conv4 = make_nn_conv(conv4)
        ## Layer 3
        #self.conv5 = make_nn_conv(conv5)
        #self.conv6 = make_nn_conv(conv6)
        ## Layer 4
        #self.conv7 = make_nn_conv(conv7)
        #self.conv8 = make_nn_conv(conv8)
        ## Layer 5
        #self.conv9 = make_nn_conv(conv9)
        #self.conv10 = make_nn_conv(conv10)
        
        ## Layer 6
        self.fc1 = nn.Linear(int(conv_features_out), 133)
        ## Layer 7
        #self.fc2 = nn.Linear(4096, 256)
        ## Layer 8
        #self.fc3 = nn.Linear(256, 133)
                
    def forward(self, x):
        ## Define forward behavior
        batch_size = x.size()[0]

        # layer 1        
        x = F.dropout(F.relu(self.conv1(x)), 0.2)
        x = F.dropout(F.max_pool2d(F.relu(self.conv2(x)), 2, 2), 0.2)
        # layer 2
        x = F.dropout(F.relu(self.conv3(x)), 0.2)
        x = F.dropout(F.max_pool2d(F.relu(self.conv4(x)), 4, 4), 0.2)
        # layer 3
        #x = F.dropout(F.relu(self.conv5(x)), 0.2)
        #x = F.dropout(F.max_pool2d(F.relu(self.conv6(x)), 4, 4), 0.2)
        # layer 4
        #x = F.dropout(F.relu(self.conv7(x)), 0.2)
        #x = F.dropout(F.max_pool2d(F.relu(self.conv8(x)), 2, 2), 0.2)
        # layer 5
        #x = F.dropout(F.relu(self.conv9(x)), 0.2)
        #x = F.dropout(F.max_pool2d(F.relu(self.conv10(x)), 2, 2), 0.2)
        
        x = x.view(batch_size, -1)
        
        #x = F.dropout(F.relu(self.fc1(x)), 0.2)
        #x = F.dropout(F.relu(self.fc2(x)), 0.2)
        #x = F.log_softmax(self.fc3(x))
        x = self.fc1(x)
        
        return x


model = Net()

if use_cuda:
    model.cuda()

32.0 16.0 16.0 4.0 768.0


In [31]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [32]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            optimizer.zero_grad()
            
            output = model(data)
            
            loss = criterion(output, target)
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss.item()*data.size(0)
        
        train_loss = train_loss/len(loaders['train'].sampler)

        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            
        valid_loss = valid_loss/len(loaders['valid'].sampler)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        if valid_loss < valid_loss_min:
            print(f'Saved model.pt, validation decresed: {valid_loss_min} => {valid_loss}')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), 'model.pt')
        
    return model

In [33]:
model = train(50, loaders, model, optimizer, 
                      criterion, use_cuda, 'model.pt')

Epoch: 1 	Training Loss: 2.983252 	Validation Loss: 1.627381
Saved model.pt, validation decresed: inf => 1.6273811308542887
Epoch: 2 	Training Loss: 0.969191 	Validation Loss: 1.246867
Saved model.pt, validation decresed: 1.6273811308542887 => 1.2468668762842814
Epoch: 3 	Training Loss: 0.841281 	Validation Loss: 1.105946
Saved model.pt, validation decresed: 1.2468668762842814 => 1.1059458653132122
Epoch: 4 	Training Loss: 0.826974 	Validation Loss: 1.021125
Saved model.pt, validation decresed: 1.1059458653132122 => 1.0211247221628825
Epoch: 5 	Training Loss: 0.840992 	Validation Loss: 1.156472
Epoch: 6 	Training Loss: 0.821332 	Validation Loss: 1.071794
Epoch: 7 	Training Loss: 0.821912 	Validation Loss: 1.040645
Epoch: 8 	Training Loss: 0.824944 	Validation Loss: 1.078237
Epoch: 9 	Training Loss: 0.825027 	Validation Loss: 0.995162
Saved model.pt, validation decresed: 1.0211247221628825 => 0.9951615842183431
Epoch: 10 	Training Loss: 0.809538 	Validation Loss: 1.029245
Epoch: 11 	Tra

In [35]:
model.load_state_dict(torch.load('model.pt'))
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

test(loaders, model, criterion, use_cuda)

Test Loss: 0.792176


Test Accuracy: 66% (400/600)
